<a href="https://colab.research.google.com/github/cornguo/colab/blob/main/Word2vec_SentSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

國教院分詞器，改寫為 Python3 相容語法

參考資料
* https://github.com/naernlp/Segmentor

安裝 CRF++

In [ ]:
!curl -L "https://drive.google.com/uc?export=download&id=0B4y35FiV1wh7QVR6VXJ5dWExSTQ" | tar xzvf -
!cd CRF++-0.58; ./configure; make; sudo make install; cd python; sudo python setup.py install
!ln -s /usr/local/lib/libcrfpp.so.0 /usr/lib/

安裝國教院分詞系統

In [ ]:
!git clone --branch python3 https://github.com/cornguo/Segmentor
!curl https://120.127.233.228/download/Segmentor/naer-segmentor-models-20160318.tar.gz -k | tar xzvf - -C Segmentor/Segmentor
!cd Segmentor; sudo python setup.py install

In [ ]:
!pip3 install gensim
!pip3 install h5py
!pip3 install gdown

In [ ]:
!wget https://cornguo.storage.googleapis.com/wiki2019tw_word2vec_cbow_d200.zip
!unzip wiki2019tw_word2vec_cbow_d200.zip

測試

In [ ]:
# -*- coding: utf-8 -*-
import json
from Segmentor import *
from gensim.models import word2vec
from gensim import models
import numpy as np
from scipy import spatial

# 載入斷詞器
segmenter = Segmentor()
# 載入 Word2vec model
model = models.Word2Vec.load('./wiki2019tw_word2vec_cbow_d200/wiki2019tw_word2vec_cbow_d200.model')

def getSentVect(query):
  # 進行斷詞
  words = segmenter.segment(query)
  wordCnt = 0
  # 計算句子的語義向量 (向量平均)
  sentVect = np.zeros((200, ), dtype='float32')
  for word in words:
    #print("processing:", word)
    if word in model.wv.index2word:
      #print("found")
      wordCnt += 1
      sentVect = np.add(sentVect, model.wv[word])

  if wordCnt > 0:
    np.divide(sentVect, wordCnt)

  #print("sentence word cnt:", wordCnt, "avg vector:", sentVect)
  return sentVect

def getSim(query1, query2):
  sentVect1 = getSentVect(query1)
  sentVect2 = getSentVect(query2)
  sim = 1 - spatial.distance.cosine(sentVect1, sentVect2)
  return sim

while True:
  # 取得輸入文字
  print("input 2 sentences, seperated by a space:")
  query = input()

  queries = query.split()
  # 取得句子的語義向量
  sim = getSim(queries[0], queries[1])
  print("similarity of", queries[0], "and", queries[1], "is", sim)
